### Forecasting of electricity consumption using a simple neural network with 2 hidden layers

In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
# from collections import defaultdict
# split a univariate sequence into samples
def split_sequenceUStep(sequence, n_steps_in):
    X, y = list(), list()
    # X, y = defaultdict(list), defaultdict(list)
    
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [3]:
# Load the data
data = pd.read_csv('electricity_demand.csv', header=0, index_col=0)

In [4]:
# Prepare the data
# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data  = data.iloc[train_size:]

In [5]:
x1,y1 = split_sequenceUStep(train_data['demand'].values, 168)
x2,y2 = split_sequenceUStep(test_data['demand'].values , 168)

In [6]:
# Convert the data into PyTorch tensors
train_inputs = torch.from_numpy(x1)
train_targets = torch.from_numpy(y1)
test_inputs = torch.from_numpy(x2)
test_targets = torch.from_numpy(y2)

In [7]:
# Define the neural network model
class ElectricityDemandForecasting(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_layer1 = torch.nn.Linear(input_size, hidden_size)
        # self.hidden_layer2 = torch.nn.Linear(hidden_size, hidden_size)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)

        # define the device to use (GPU or CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, input):
        hidden = torch.relu(self.hidden_layer1(input))
        # hidden = torch.relu(self.hidden_layer2(hidden))
        output = self.output_layer(hidden)
        return output

In [8]:
# class ElectricityDemandForecasting(torch.nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(ElectricityDemandForecasting, self).__init__()
#         self.seq_model = torch.nn.Sequential(
#             torch.nn.Linear(input_size, hidden_size),
#             torch.nn.ReLU(),
#             torch.nn.Linear(hidden_size, output_size),
#             torch.nn.Softmax(dim=1)
#         )
        
#     def forward(self, x):
#         out = self.seq_model(x)
#         return out

In [9]:
# Initialize the model and optimizer
model = ElectricityDemandForecasting(input_size=168, hidden_size=200, output_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Train the model
for epoch in range(100):
    # Forward pass
    train_predictions = model(train_inputs.float())
    train_loss = torch.nn.MSELoss()(train_predictions.float().squeeze(), train_targets.float())

    # Backward pass
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # Print the training loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Train Loss: {train_loss.item()}')

Epoch 10, Train Loss: 0.012834024615585804
Epoch 20, Train Loss: 0.005190882366150618
Epoch 30, Train Loss: 0.004092255141586065
Epoch 40, Train Loss: 0.0033377904910594225
Epoch 50, Train Loss: 0.0028885011561214924
Epoch 60, Train Loss: 0.0025810266379266977
Epoch 70, Train Loss: 0.0023641642183065414
Epoch 80, Train Loss: 0.0021668013650923967
Epoch 90, Train Loss: 0.001996682956814766
Epoch 100, Train Loss: 0.0018512536771595478


In [11]:
# Evaluate the model on the test set
test_predictions = model(test_inputs.float())
test_loss = torch.nn.MSELoss()(test_predictions.float().squeeze(), test_targets.float())
print(f'Test Loss: {test_loss.item()}')

Test Loss: 0.0012537797447293997


In [12]:
# Save the trained model
torch.save(model.state_dict(), "model.pth")

In [13]:
# # Load the saved model
# model.load_state_dict(torch.load("model.pth"))
# model.eval()

# # Predict demand for new input
# with torch.no_grad():
#     predicted_demand = model(new_input)


In [14]:
# Convert the test predictions and test output to NumPy arrays
test_targets = test_targets.detach().numpy()
test_predictions = test_predictions.detach().numpy()


In [15]:
test_targets_df = pd.DataFrame(test_targets, columns=['Test Demand'])
test_targets_df.to_csv('test_targets.csv', index=False)
test_predictions_df = pd.DataFrame(test_predictions, columns=['Predicted Demand'])
test_predictions_df.to_csv('test_predictions.csv', index=False)

In [16]:
time_df = pd.DataFrame({'LoadLevel': pd.date_range(start='2023-05-04 00:00:00', periods=len(test_predictions_df), freq='H')})
frames = [time_df, test_predictions_df, test_targets_df]
result = pd.concat(frames, axis=1).set_index('LoadLevel')

In [17]:
source = result.stack().reset_index().rename(columns={'level_1':'Demand', 0:'Value'})

In [18]:
import altair as alt

lines = (
    alt.Chart(source)
    .mark_line()
    .encode(x="LoadLevel", y="Value", color="Demand")
).properties(width=1500, height=500)
lines.save('Plot.html', embed_options={'renderer':'svg'})
